# Local search - Flowshop problem

### About this notebook
This notebook contains a hands-on the flowshop problem. We will focus on implementing some of the most known local search algorithms in order to solve this problem.

### Used ressources
- [Benchmarks for Basic Scheduling Problems](http://mistic.heig-vd.ch/taillard/articles.dir/Taillard1993EJOR.pdf)
- [Implement Simulated annealing in Python](https://medium.com/swlh/how-to-implement-simulated-annealing-algorithm-in-python-ab196c2f56a0)
- [Introduction to Hill Climbing](https://www.geeksforgeeks.org/introduction-hill-climbing-artificial-intelligence/)

In [1]:
import numpy as np
import matplotlib as plt
import itertools
import time
import pandas as pd
import math
import random

In [2]:
def evaluate_sequence(sequence, processing_times):
    _, num_machines = processing_times.shape
    num_jobs = len(sequence)
    completion_times = np.zeros((num_jobs, num_machines))
    
    # Calculate the completion times for the first machine
    completion_times[0][0] = processing_times[sequence[0]][0]
    for i in range(1, num_jobs):
        completion_times[i][0] = completion_times[i-1][0] + processing_times[sequence[i]][0]
    
    # Calculate the completion times for the remaining machines
    for j in range(1, num_machines):
        completion_times[0][j] = completion_times[0][j-1] + processing_times[sequence[0]][j]
        for i in range(1, num_jobs):
            completion_times[i][j] = max(completion_times[i-1][j], completion_times[i][j-1]) + processing_times[sequence[i]][j]
    
    # Return the total completion time, which is the completion time of the last job in the last machine
    return completion_times[num_jobs-1][num_machines-1]

In [3]:
# Generate a random example to work with 7 jobs and 2 machines
rnd_data = np.random.randint(size=(15,10), low=5, high=120)
permutation = np.random.permutation(10).tolist()
print(rnd_data, "\n")
print('Initial solution:', permutation, "\n")

Cmax = evaluate_sequence(permutation, rnd_data)
print(f'Makespan: {Cmax}')

[[ 52  92  96  78  37  26   6   8 116 114]
 [ 92  74  78  92  19 101  82   6  92  66]
 [ 14  73  33 110  95  47  57  94  78  74]
 [ 21  32 112  61  71  11  42  20  52  72]
 [ 64  14  59  38  14  91  56  92  28   9]
 [  8  19  12  72  36 108 105 119  71  86]
 [  8 108  59  53 103  74  97  36  81 110]
 [ 13  63  84 100  25  62  48  45  18  37]
 [ 56  40  41  13  88  63  30   6  60  12]
 [114  92   5  27  73  90  13   8  54  98]
 [ 40  56  56  41  28  95  51  71  99  16]
 [111  19 103 118  66  73 116  96  89  68]
 [  8  61   9  78  44 111 115  14  97  76]
 [ 37  20  23  65  97  42  32  26  39  64]
 [ 47  54  56   9  34  57  78  81  80  20]] 

Initial solution: [3, 5, 2, 8, 1, 9, 6, 4, 7, 0] 

Makespan: 1457.0


## Generating neighbours

#### Swaping methods

In [4]:
def swap(solution, i, k):
    temp = solution[k]
    solution[k] = solution[i]
    solution[i] = temp
    return solution

In [5]:
def random_swap(solution, processing_times):
    i = np.random.choice(list(solution))
    k = np.random.choice(list(solution))
    # Generating two different random positions
    while (i == k):
        k = np.random.choice(list(solution))
    # Switch between job i and job k in the given sequence
    neighbor = solution.copy()
    return swap(neighbor, i, k), evaluate_sequence(neighbor, processing_times)

In [6]:
print("Original solution: ", permutation)
print("Makespan: ", Cmax)

start_time = time.time()
neighbor, neighbor_cmax = random_swap(permutation, rnd_data)
elapsed_time = time.time() - start_time

print("\nNeighboor solution: ", neighbor)
print("Makespan: ", neighbor_cmax)
print("\nElapsed time:", elapsed_time, "seconds")

Original solution:  [3, 5, 2, 8, 1, 9, 6, 4, 7, 0]
Makespan:  1457.0

Neighboor solution:  [3, 5, 2, 8, 4, 9, 6, 1, 7, 0]
Makespan:  1463.0

Elapsed time: 0.0 seconds


In [7]:
def best_first_swap(solution, processing_times):
    num_jobs = len(solution)
    best_cmax = evaluate_sequence(solution, processing_times)
    best_neighbor = solution.copy()
    for k1 in range(num_jobs):
        for k2 in range(k1+1, num_jobs):
            neighbor = solution.copy()
            neighbor = swap(neighbor,k1,k2)
            cmax = evaluate_sequence(neighbor, processing_times)
            if cmax < best_cmax:
                best_neighbor = neighbor
                best_cmax = cmax
                return best_neighbor, best_cmax
    return best_neighbor, best_cmax

In [8]:
print("Original solution: ", permutation)
print("Makespan: ", Cmax)

start_time = time.time()
neighbor, neighbor_cmax = best_first_swap(permutation, rnd_data)
elapsed_time = time.time() - start_time

print("\nNeighboor solution: ", neighbor)
print("Makespan: ", neighbor_cmax)
print("\nElapsed time:", elapsed_time, "seconds")

Original solution:  [3, 5, 2, 8, 1, 9, 6, 4, 7, 0]
Makespan:  1457.0

Neighboor solution:  [5, 3, 2, 8, 1, 9, 6, 4, 7, 0]
Makespan:  1393.0

Elapsed time: 0.0 seconds


In [9]:
def best_swap(solution, processing_times):
    num_jobs = len(solution)
    best_cmax = np.Infinity
    for k1 in range(num_jobs):
        for k2 in range(k1+1, num_jobs):
            neighbor = solution.copy()
            neighbor = swap(neighbor,k1,k2)
            cmax = evaluate_sequence(neighbor, processing_times)
            if cmax < best_cmax:
                best_neighbor = neighbor
                best_cmax = cmax
    return best_neighbor, best_cmax

In [10]:
print("Original solution: ", permutation)
print("Makespan: ", Cmax)

start_time = time.time()
neighbor, neighbor_cmax = best_swap(permutation, rnd_data)
elapsed_time = time.time() - start_time

print("\nNeighboor solution: ", neighbor)
print("Makespan: ", neighbor_cmax)
print("\nElapsed time:", elapsed_time, "seconds")

Original solution:  [3, 5, 2, 8, 1, 9, 6, 4, 7, 0]
Makespan:  1457.0

Neighboor solution:  [3, 5, 6, 8, 1, 9, 2, 4, 7, 0]
Makespan:  1379.0

Elapsed time: 0.009977102279663086 seconds


In [11]:
def best_swaps(solution, processing_times):
    num_jobs = len(solution)
    cmax = evaluate_sequence(solution, processing_times)
    bests = []
    for k1 in range(num_jobs):
        for k2 in range(k1+1, num_jobs):
            neighbor = solution.copy()
            swap(neighbor,k1,k2)
            neighbor_cmax = evaluate_sequence(neighbor, processing_times)
            if neighbor_cmax < cmax:
                bests.append((neighbor, neighbor_cmax))
    return bests

In [12]:
print("Original solution: ", permutation)
print("Makespan: ", Cmax)

start_time = time.time()
bests= best_swaps(permutation, rnd_data)
elapsed_time = time.time() - start_time

print("\n\nNeighboors:", bests)

Original solution:  [3, 5, 2, 8, 1, 9, 6, 4, 7, 0]
Makespan:  1457.0


Neighboors: [([5, 3, 2, 8, 1, 9, 6, 4, 7, 0], 1393.0), ([2, 5, 3, 8, 1, 9, 6, 4, 7, 0], 1413.0), ([8, 5, 2, 3, 1, 9, 6, 4, 7, 0], 1385.0), ([4, 5, 2, 8, 1, 9, 6, 3, 7, 0], 1436.0), ([3, 8, 2, 5, 1, 9, 6, 4, 7, 0], 1456.0), ([3, 5, 8, 2, 1, 9, 6, 4, 7, 0], 1401.0), ([3, 5, 1, 8, 2, 9, 6, 4, 7, 0], 1409.0), ([3, 5, 9, 8, 1, 2, 6, 4, 7, 0], 1455.0), ([3, 5, 6, 8, 1, 9, 2, 4, 7, 0], 1379.0), ([3, 5, 4, 8, 1, 9, 6, 2, 7, 0], 1437.0), ([3, 5, 0, 8, 1, 9, 6, 4, 7, 2], 1400.0), ([3, 5, 2, 1, 8, 9, 6, 4, 7, 0], 1416.0), ([3, 5, 2, 9, 1, 8, 6, 4, 7, 0], 1442.0), ([3, 5, 2, 4, 1, 9, 6, 8, 7, 0], 1428.0), ([3, 5, 2, 7, 1, 9, 6, 4, 8, 0], 1447.0), ([3, 5, 2, 0, 1, 9, 6, 4, 7, 8], 1400.0), ([3, 5, 2, 8, 0, 9, 6, 4, 7, 1], 1448.0), ([3, 5, 2, 8, 1, 6, 9, 4, 7, 0], 1451.0), ([3, 5, 2, 8, 1, 7, 6, 4, 9, 0], 1448.0), ([3, 5, 2, 8, 1, 9, 0, 4, 7, 6], 1430.0), ([3, 5, 2, 8, 1, 9, 6, 0, 7, 4], 1424.0)]


#### Insertion methods

In [13]:
def random_insertion(solution, processing_times):
    i = np.random.choice(list(solution))
    k = np.random.choice(list(solution))
    while (i == k):
        k = np.random.choice(list(solution))
    neighbor = solution.copy()
    neighbor.remove(solution[i])
    neighbor.insert(k, solution[i])
    return neighbor, evaluate_sequence(neighbor, processing_times)

In [14]:
print("Original solution: ", permutation)
print("Makespan: ", Cmax)

start_time = time.time()
neighbor, neighbor_cmax = random_insertion(permutation, rnd_data)
elapsed_time = time.time() - start_time

print("\nNeighboor solution: ", neighbor)
print("Makespan: ", neighbor_cmax)
print("\nElapsed time:", elapsed_time, "seconds")

Original solution:  [3, 5, 2, 8, 1, 9, 6, 4, 7, 0]
Makespan:  1457.0

Neighboor solution:  [3, 5, 2, 8, 1, 7, 9, 6, 4, 0]
Makespan:  1484.0

Elapsed time: 0.0005185604095458984 seconds


In [15]:
def best_insertion(solution, processing_times):
    num_jobs = len(solution)
    best_cmax = np.Infinity
    for k1 in range(num_jobs):
        s = solution.copy()
        s_job = solution[k1]
        s.remove(s_job)
        for k2 in range(num_jobs):
            if k1 != k2:
                neighbor = s.copy()
                neighbor.insert(k2, s_job)
                cmax = evaluate_sequence(neighbor, processing_times)
                if cmax < best_cmax:
                    best_neighbor = neighbor
                    best_cmax = cmax
    return best_neighbor, best_cmax

In [16]:
print("Original solution: ", permutation)
print("Makespan: ", Cmax)

start_time = time.time()
neighbor, neighbor_cmax = best_insertion(permutation, rnd_data)
elapsed_time = time.time() - start_time

print("\nNeighboor solution: ", neighbor)
print("Makespan: ", neighbor_cmax)
print("\nElapsed time:", elapsed_time, "seconds")

Original solution:  [3, 5, 2, 8, 1, 9, 6, 4, 7, 0]
Makespan:  1457.0

Neighboor solution:  [5, 2, 8, 3, 1, 9, 6, 4, 7, 0]
Makespan:  1332.0

Elapsed time: 0.02892279624938965 seconds


In [17]:
def best_edge_insertion(solution, processing_times):
    num_jobs = len(solution)
    best_cmax = np.Infinity
    for k1 in range(num_jobs-1):
        s = solution.copy()
        s_job1 = s[k1] 
        s_job2 = s[k1+1]
        s.remove(s_job1)
        s.remove(s_job2)
        for k2 in range(num_jobs-1):
            if(k1 != k2):
                neighbor = s.copy()
                neighbor.insert(k2, s_job1)
                neighbor.insert(k2+1, s_job2)
                cmax = evaluate_sequence(neighbor, processing_times)
                if cmax < best_cmax:
                    best_neighbor = neighbor
                    best_cmax = cmax
    return best_neighbor, best_cmax

In [18]:
print("Original solution: ", permutation)
print("Makespan: ", Cmax)

start_time = time.time()
neighbor, neighbor_cmax = best_edge_insertion(permutation, rnd_data)
elapsed_time = time.time() - start_time

print("\nNeighboor solution: ", neighbor)
print("Makespan: ", neighbor_cmax)
print("\nElapsed time:", elapsed_time, "seconds")

Original solution:  [3, 5, 2, 8, 1, 9, 6, 4, 7, 0]
Makespan:  1457.0

Neighboor solution:  [5, 2, 3, 8, 1, 9, 6, 4, 7, 0]
Makespan:  1363.0

Elapsed time: 0.01795339584350586 seconds


## Hill Climbing

#### Simple Hill climbing

It examines the neighboring nodes one by one and selects the first neighboring node which optimizes the current cost as the next node. 

In [19]:
def simple_hill_climbing(solution, processing_times, nb_iter=10000):
    x = solution
    cmax = evaluate_sequence(solution, processing_times)
    iterations = 0
    while iterations < nb_iter:
        best_neighbor, best_cmax  = best_first_swap(x, processing_times)
        if best_cmax == cmax:
            return best_neighbor, best_cmax, iterations
        x = best_neighbor
        cmax = best_cmax
        iterations += 1
    return x, cmax, iterations

In [20]:
print("Original solution: ", permutation)
print("Makespan: ", Cmax)

start_time = time.time()
best_solution, best_cmax, nb_iterations = simple_hill_climbing(permutation, rnd_data)
elapsed_time = time.time() - start_time

print("\nBest solution found: ", best_solution)
print("Makespan: ", best_cmax)
print("Number of iterations: ", nb_iterations)
print("\nElapsed time:", elapsed_time, "seconds")

Original solution:  [3, 5, 2, 8, 1, 9, 6, 4, 7, 0]
Makespan:  1457.0

Best solution found:  [5, 3, 6, 9, 2, 0, 1, 8, 7, 4]
Makespan:  1228.0
Number of iterations:  10

Elapsed time: 0.08487796783447266 seconds


#### Steepest-Ascent Hill climbing

In [21]:
def steepest_ascent_hill_climbing(solution, processing_times, nb_iter=1000):
    x = solution
    cmax = evaluate_sequence(solution, processing_times)
    iterations = 0
    while iterations < nb_iter:
        best_neighbor, best_cmax = best_swap(solution, processing_times)
        if best_cmax > cmax:
            return x, cmax
        else:
            x = best_neighbor
            cmax = best_cmax
            iterations += 1
    return best_neighbor, best_cmax, iterations

In [22]:
print("Original solution: ", permutation)
print("Makespan: ", Cmax)

start_time = time.time()
best_solution, best_cmax, nb_iterations = steepest_ascent_hill_climbing(permutation, rnd_data)
elapsed_time = time.time() - start_time

print("\nBest solution found: ", best_solution)
print("Makespan: ", best_cmax)
print("Number of iterations: ", nb_iterations)
print("\nElapsed time:", elapsed_time, "seconds")

Original solution:  [3, 5, 2, 8, 1, 9, 6, 4, 7, 0]
Makespan:  1457.0

Best solution found:  [3, 5, 6, 8, 1, 9, 2, 4, 7, 0]
Makespan:  1379.0
Number of iterations:  1000

Elapsed time: 11.794685363769531 seconds


#### Stochastic Hill climbing

In [23]:
def stochastic_hill_climbing(solution, processing_times, nb_iter=1000):
    x = solution
    cmax = evaluate_sequence(solution, processing_times)
    iterations = 0
    while iterations < nb_iter:
        best_neighbours  = best_swaps(x, processing_times)
        if len(best_neighbours) == 0:
            return x, cmax, iterations
        i = random.randint(0,len(best_neighbours) - 1)
        best_neighbor, best_cmax = best_neighbours[i]
        if best_cmax > cmax:
            return x, cmax, iterations
        x = best_neighbor
        cmax = best_cmax
        iterations += 1
    return best_neighbor, best_cmax, iterations

In [24]:
print("Original solution: ", permutation)
print("Makespan: ", Cmax)

start_time = time.time()
best_solution, best_cmax, nb_iterations = stochastic_hill_climbing(permutation, rnd_data)
elapsed_time = time.time() - start_time

print("\nBest solution found: ", best_solution)
print("Makespan: ", best_cmax)
print("Number of iterations: ", nb_iterations)
print("\nElapsed time:", elapsed_time, "seconds")

Original solution:  [3, 5, 2, 8, 1, 9, 6, 4, 7, 0]
Makespan:  1457.0

Best solution found:  [5, 3, 6, 9, 2, 0, 1, 8, 7, 4]
Makespan:  1228.0
Number of iterations:  12

Elapsed time: 0.18952059745788574 seconds


## Simulated annealing

In [25]:
def get_neighbor(solution, processing_times):
    i = random.randint(0, 5)
    # Swapping methods
    if i == 0:
        neighbor, cost = random_swap(solution, processing_times)
    elif i == 1:
        neighbor, cost = best_swap(solution, processing_times)
    elif i == 2:
        neighbor, cost = best_first_swap(solution, processing_times)
    # Insertion methods
    elif i == 3:
        neighbor, cost = random_insertion(solution, processing_times)
    elif i == 4:
        neighbor, cost = best_edge_insertion(solution, processing_times)
    else:
        neighbor, cost = best_insertion(solution, processing_times)
    return neighbor, cost

In [26]:
def simulated_annealing(initial_solution, processing_times, initial_temp=90, final_temp=1, alpha=0.1):
    current_temp = initial_temp
    current_solution = initial_solution.copy()
    current_cost = evaluate_sequence(initial_solution, processing_times)
    while current_temp > final_temp:
        neighbor, neighbor_cost  = get_neighbor(current_solution, processing_times)
        cost_diff = current_cost - neighbor_cost
        if cost_diff > 0:
            current_solution = neighbor
            current_cost = neighbor_cost
        else:
            if random.uniform(0, 1) < math.exp(-cost_diff / current_temp):
                solution = neighbor
        current_temp -= alpha
    return current_solution, current_cost

In [27]:
start_time = time.time()
best_solution, best_solution_length = simulated_annealing(permutation, rnd_data)
elapsed_time = time.time() - start_time

print("Best solution found: ", best_solution)
print("Makespan: ", evaluate_sequence(best_solution, rnd_data))
print("Elapsed time:", elapsed_time, "seconds")

Best solution found:  [5, 3, 6, 9, 2, 1, 7, 0, 4, 8]
Makespan:  1228.0
Elapsed time: 11.03171181678772 seconds
